In [1]:
from accounts import MasterAccount, Account, MultSigAccount
from transactions import Transaction, MultiSigTransaction
from os import urandom
from database import WalletDB
from blockcypher import get_address_details

In [5]:
class Wallet(MasterAccount):
    
    def __init__(self, depth, fingerprint, index, chain_code, private_key, db_user="neo4j",db_password="wallet", testnet = False):
    
        self.db = WalletDB( "neo4j://localhost:7687" ,db_user ,db_password )
        super().__init__( depth, fingerprint, index, chain_code, private_key,testnet)
    
        #default_path = "m/0H/0H/"
    
    @classmethod
    def get_i(self, index):
        if not index:
            print(f"get i: {index}")
            i = int.from_bytes(urandom(4),"big")
            i = i & 0x7fffffff
            if i < (2**31-1):print("true")
        else:
            if index <= (2**31-1):
                i = index
            else:
                raise Exception (f"index must be less than {2**31-1} ")
        return i

    #@classmethod
    def create_receiving_address(self, addr_type = "p2pkh",index=None):
        receiving_path = "m/0H/2H/"
        i = self.get_i(index)
        path = receiving_path + str(i)
        print(f"Path: {path}")
        receiving_xtended_acc = self.get_child_from_path(path)
        account = Account(int.from_bytes(receiving_xtended_acc.private_key,"big"),addr_type, self.testnet )
        self.db.new_address(account.address,i,False)
        return account

    #@classmethod
    def create_change_address(self,addr_type = "p2wpkh", index=None):
        change_path = "m/0H/1H/"
        i = self.get_i(index)
        path = change_path + str(i)
        print(f"Path: {path}")
        change_xtended_acc = self.get_child_from_path(path)
        account = Account(int.from_bytes(change_xtended_acc.private_key,"big"),addr_type, self.testnet )
        self.db.new_address(account.address,i,True)
        return account
    
    def get_utxos(self):
        return self.db.look_for_coins()
        
    def get_balance(self):
        coins = self.get_utxos()
        balance = 0
        for coin in coins:
            balance += coin["coin.amount"]
        return balance

    def update_balance(self):
        addresses = self.db.get_all_addresses()
        
        if self.testnet: coin_symbol = "btc-testnet"
        else: coin_symbol = "btc"
            
        for address in addresses:
            
            addr_info = get_address_details(address["addr.address"], coin_symbol = coin_symbol, unspent_only=True)
            
            if addr_info["unconfirmed_n_tx"] > 0:
                for utxo in addr_info["unconfirmed_txrefs"]:
                    if not self.db.exists_utxo( utxo["tx_hash"], utxo["tx_output_n"], False):
                        print("new unconfirmed UTXO")
                        self.db.new_utxo(utxo["address"],utxo["tx_hash"],utxo["tx_output_n"],utxo["value"],False)
            
            if addr_info["n_tx"] - addr_info["unconfirmed_n_tx"] > 0 :
                for utxo in addr_info["txrefs"]:
                    if not self.db.exists_utxo( utxo["tx_hash"], utxo["tx_output_n"], True):
                        print("new confirmed UTXO")
                        self.db.new_utxo(addr_info["address"],utxo["tx_hash"],utxo["tx_output_n"],utxo["value"],True)
      
        return self.get_balance()
    
    #@classmethod
    #def create_multisig_account(m,public_key_list,account,addr_type="p2sh", testnet = False, segwit=True):
    #    n = len(public_key_list)
    #    return MultiSigTransaction(m, n, int.from_bytes(account.private_key,"big"), public_key_list, addr_type, testnet, segwit)

    #@classmethod
    def send(self, input_id_list, to_address, amount):
        """
        to_address can be a single address or a list.
        amount can be an integer or a list of integers.
        If they are lists, they must be ordered in the same way. address 1 will e sent amount 1, 
        adrress 2 will be sent amount2.. adress n will be sent amount n.
        """
        raise NotImplementedError
        
    
    

In [3]:
words = "engine over neglect science fatigue dawn axis parent mind man escape era goose border invest slab relax bind desert hurry useless lonely frozen morning"
my_wallet = Wallet.recover_from_words(words, 256, "RobertPauslon",True)
my_wallet

tprv8ZgxMBicQKsPfQJYjuFAso9x6STzmUdMh5U8CQqqQUTgtQHBHCq4C7FseeeZg15L16UeSwbrLwJRTXNPQsJQwqvbBA11sn4M6c3jR1LwAQP

In [16]:
change_addr = my_wallet.create_change_address()

before: None
get i: None
true
Path: m/0H/1H/1920255512
<Record a=<Path start=<Node id=1 labels={'address'} properties={'address': 'tb1q33psda8sztpq3lc27jyqy5mx57c606x2w70n5h', 'type': 'change', 'acc_index': 1920255512, 'created': 1586288905624}> end=<Node id=1 labels={'address'} properties={'address': 'tb1q33psda8sztpq3lc27jyqy5mx57c606x2w70n5h', 'type': 'change', 'acc_index': 1920255512, 'created': 1586288905624}> size=0>>


In [17]:
receiving_addr = my_wallet.create_receiving_address()

before: None
get i: None
true
Path: m/0H/2H/1091795145
<Record a=<Path start=<Node id=17 labels={'address'} properties={'address': 'n4nzNicx9sYzH6kHa6cVVVSD8nbS7jHFfd', 'type': 'recipient', 'acc_index': 1091795145, 'created': 1586288909972}> end=<Node id=17 labels={'address'} properties={'address': 'n4nzNicx9sYzH6kHa6cVVVSD8nbS7jHFfd', 'type': 'recipient', 'acc_index': 1091795145, 'created': 1586288909972}> size=0>>


In [18]:
change_addr.address

'tb1q33psda8sztpq3lc27jyqy5mx57c606x2w70n5h'

In [19]:
receiving_addr.address


'n4nzNicx9sYzH6kHa6cVVVSD8nbS7jHFfd'

In [4]:
my_wallet.update_balance()

<Record p=<Path start=<Node id=6 labels={'utxo'} properties={'transaction_id': '865bb2159bf4303e62cc6db1a55cdd5faf7cf553dddaac9436e79e2f58127900', 'out_index': 1, 'amount': 1000000, 'address': 'tb1q33psda8sztpq3lc27jyqy5mx57c606x2w70n5h', 'spent': False, 'local_index': 2, 'confirmed': False}> end=<Node id=1 labels={'address'} properties={'address': 'tb1q33psda8sztpq3lc27jyqy5mx57c606x2w70n5h', 'type': 'change', 'acc_index': 1920255512, 'created': 1586288905624}> size=1>>
<Record p=<Path start=<Node id=7 labels={'utxo'} properties={'transaction_id': '0715c2c52bb1da89d4fc9df846b1eeac03df3c9155b8ded50889955dcb8d9f55', 'out_index': 1, 'amount': 10000, 'address': 'tb1q33psda8sztpq3lc27jyqy5mx57c606x2w70n5h', 'spent': False, 'local_index': 3, 'confirmed': False}> end=<Node id=1 labels={'address'} properties={'address': 'tb1q33psda8sztpq3lc27jyqy5mx57c606x2w70n5h', 'type': 'change', 'acc_index': 1920255512, 'created': 1586288905624}> size=1>>
<Record p=<Path start=<Node id=8 labels={'utxo'} p

2082000

In [19]:
type(receiving_addr.address)

NameError: name 'receiving_addr' is not defined

In [25]:

ddd = get_address_details('n4nzNicx9sYzH6kHa6cVVVSD8nbS7jHFfd', coin_symbol = "btc-testnet", unspent_only=True)

In [26]:
ddd

{'address': 'n4nzNicx9sYzH6kHa6cVVVSD8nbS7jHFfd',
 'total_received': 50000,
 'total_sent': 0,
 'balance': 50000,
 'unconfirmed_balance': 0,
 'final_balance': 50000,
 'n_tx': 1,
 'unconfirmed_n_tx': 0,
 'final_n_tx': 1,
 'txrefs': [{'tx_hash': '7e466d74ee32e8fe611c11ac86b9b7c39996ff9476ea09875a06f02ecab42ada',
   'block_height': 1696602,
   'tx_input_n': -1,
   'tx_output_n': 1,
   'value': 50000,
   'ref_balance': 50000,
   'spent': False,
   'confirmations': 2,
   'confirmed': datetime.datetime(2020, 4, 7, 21, 47, 41, tzinfo=tzutc()),
   'double_spend': False}],
 'tx_url': 'https://api.blockcypher.com/v1/btc/test3/txs/',
 'unconfirmed_txrefs': []}

In [27]:
ddd["txrefs"][0]["address"]

KeyError: 'address'

In [28]:
ddd["txrefs"][0]["value"]

50000

In [29]:
ddd["txrefs"][0]["tx_hash"]

'7e466d74ee32e8fe611c11ac86b9b7c39996ff9476ea09875a06f02ecab42ada'

In [30]:
ddd["txrefs"][0]["tx_output_n"]

1

In [31]:
ddd["unconfirmed_n_tx"]

0

In [38]:
ddd2 = ddd["txrefs"]

In [40]:
ddd2[0]

{'tx_hash': '7e466d74ee32e8fe611c11ac86b9b7c39996ff9476ea09875a06f02ecab42ada',
 'block_height': 1696602,
 'tx_input_n': -1,
 'tx_output_n': 1,
 'value': 50000,
 'ref_balance': 50000,
 'spent': False,
 'confirmations': 2,
 'confirmed': datetime.datetime(2020, 4, 7, 21, 47, 41, tzinfo=tzutc()),
 'double_spend': False}

In [5]:
x = [{'coin.amount': 22000, 'coin.confirmed': True}]

In [6]:
x[0]

{'coin.amount': 22000, 'coin.confirmed': True}